# NOTE: Some parts of this are untested or deprecated. This notebook will be reworked in the future or removed.

As found in the [previous notebook](01_Data_Exploration.ipynb), the column "diagnosis" is missing from the pkl files, because tokenization didn't run. Let's see by running some experiments using the author's code what files and columns are missing in particular. So, we can narrow down the problem and fix it.

## Imports

In [1]:
import subprocess
from pathlib import Path
import datetime

## Functions

In [2]:
# Function to convert datetime isoformat to POSIX compatible file name
def iso_to_posix(iso):
    return iso.replace(':', '-').replace('.', '-')

def get_time():
    dtime = datetime.datetime.now().isoformat()
    return iso_to_posix(dtime)

In [3]:
get_time()

'2024-04-09T23-28-58-037654'

## Experiments

### Pretraining CodeEmb with word2vec

In [4]:
def listify_command(command):
    return [ch for ch in command.split(" ") if ch != ""]

def run_command(command):
    # Open the subprocess
    if isinstance(command, str):
        command = listify_command(command)
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1  # line buffered
    )

    # Continuously print stdout
    with process.stdout:
        for line in iter(process.stdout.readline, ''):
            print(line, end='')

    # Wait for the process to finish
    process.wait()

In [9]:
input_path = "/data/DescEmb/output_predict"
data = "mimiciii"
model = "codeemb"
WORLDSIZE = 1
value_mode = "NV"
run = get_time()
checkpoint_prefix = f"{model}_{data}_{value_mode}_run_{run}_checkpoint_best"
checkpoint_path = f"checkpoints"
task = "w2v"

command = f"python ../DescEmb/main.py \
    --distributed_world_size {WORLDSIZE} \
    --input_path {input_path} \
    --src_data {data} \
    --task {task} \
    --model {model} \
    --value_mode {value_mode} \
    --save_dir {checkpoint_path} \
    --save_prefix {checkpoint_prefix}"
#run_command(command)

In [10]:
! $command

[2024-04-09 23:41:00,244][trainers.word2vec_trainer][INFO] - epoch: 0, loss: 1.386
[2024-04-09 23:41:00,245][trainers.word2vec_trainer][INFO] - Saving checkpoint to checkpoints/codeemb_mimiciii_NV_run_2024-04-09T23-40-46-157112_checkpoint_best_best.pt
[2024-04-09 23:41:00,275][trainers.word2vec_trainer][INFO] - Finished saving checkpoint to checkpoints/codeemb_mimiciii_NV_run_2024-04-09T23-40-46-157112_checkpoint_best_best.pt
[2024-04-09 23:41:00,419][trainers.word2vec_trainer][INFO] - epoch: 1, loss: 1.386
Validation AUROC increased (0.000000 --> -1.386221)
[2024-04-09 23:41:00,419][trainers.word2vec_trainer][INFO] - Saving checkpoint to checkpoints/codeemb_mimiciii_NV_run_2024-04-09T23-40-46-157112_checkpoint_best_best.pt
[2024-04-09 23:41:00,450][trainers.word2vec_trainer][INFO] - Finished saving checkpoint to checkpoints/codeemb_mimiciii_NV_run_2024-04-09T23-40-46-157112_checkpoint_best_best.pt
[2024-04-09 23:41:00,589][trainers.word2vec_trainer][INFO] - epoch: 2, loss: 1.386
Valid

### Pretrain a DescEmb-RNN model

In [21]:
input_path = "/data/DescEmb/output_predict"
data = "mimiciii"
model = "descemb_rnn"
WORLDSIZE = 1
value_mode = "NV"
run = get_time()
checkpoint_prefix = f"{model}_{data}_{value_mode}_run_{run}_checkpoint_best"
checkpoint_path = f"checkpoints"
task = "mlm"
ratio = 100

command = f"CUDA_VISIBLE_DEVICES=0 python ../DescEmb/main.py \
    --distributed_world_size {WORLDSIZE} \
    --input_path {input_path} \
    --src_data {data} \
    --ratio 100 \
    --task {task} \
    --model {model} \
    --value_mode {value_mode} \
    --save_dir {checkpoint_path} \
    --save_prefix {checkpoint_prefix}"

In [22]:
! $command

[2024-04-10 00:04:52,759][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': None,
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/data/DescEmb/output_predict',
 'load_pretrained_weights': False,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'descemb_rnn',
 'model_path': None,
 'n_epochs': 1000,
 'patience': 5,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': None,
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'descemb_rnn_mimiciii_NV_run_2024-04-10T00-04-46-744492_checkpoint_best',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'mlm',
 'transfer': False,
 'valid_subsets': [],
 'value_mode': 'NV'}
/media/netzdose/constantin/Uni/UIllinois/courses/CS598_DL/project/project_

### Fine-tune a pre-trained DescEmb model

In [ ]:
input_path = "/data/DescEmb/output_predict"
data = "mimiciii"
model = "codeemb"
WORLDSIZE = 1
value_mode = "NV"
run = get_time()
checkpoint_prefix = f"{model}_{data}_{value_mode}_run_{run}_checkpoint_best"
checkpoint_path = f"checkpoints"

command = f"python ../DescEmb/main.py \
    --distributed_world_size {WORLDSIZE} \
    --input_path {input_path} \
    --src_data {data} \
    --task w2v \
    --model {model} \
    --value_mode {value_mode} \
    --save_dir {checkpoint_path} \
    --save_prefix {checkpoint_prefix}"
run_command(command)

In [ ]:
python main.py \
    --distributed_world_size $WORLDSIZE \
    --input_path /path/to/data \
    --model_path /path/to/model.pt \
    --transfer \
    --model ehr_model \
    --embed_model $embed_model \
    --pred_model rnn \
    --src_data $data \
    --ratio $ratio \
    --value_mode $value \
    --task $task

In [1]:
arg_dict = {
    'distributed_world_size': 1,
    'input_path': "None",
    'model_path': "None",
    'save_dir': "../project_DLH/checkpoints",
    'save_prefix': "checkpoint",
    'patience': 5,
    'disable_validation': "None",
    'src_data': "None",
    'eval_data': "None",
    'value_mode': "NV",
    'fold': "None",
    'valid_subsets': "valid, test",
    'task': "readmission",
    'seed': 1,
    'ratio': "100",
    'n_epochs': 1000,
    'lr': 0.0001,
    'batch_size': 128,
    'enc_embed_dim': 128,
    'enc_hidden_dim': 256,
    'rnn_layer': 1,
    'dropout': 0.3,
    'pred_embed_dim': 128,
    'pred_hidden_dim': 256,
    'max_event_len': 150,
    'mlm_prob': 0.3,
    'load_pretrained_weights': "None",
    'transfer': "None",
    'model': "None",
    'embed_model': "None",
    'pred_model': "None",
    'bert_model': "bert_tiny",
    'init_bert_params': "None",
    'init_bert_params_with_freeze': "None"
}


In [2]:
arg_dict

{'distributed_world_size': 1,
 'input_path': 'None',
 'model_path': 'None',
 'save_dir': '../project_DLH/checkpoints',
 'save_prefix': 'checkpoint',
 'patience': 5,
 'disable_validation': 'None',
 'src_data': 'None',
 'eval_data': 'None',
 'value_mode': 'NV',
 'fold': 'None',
 'valid_subsets': 'valid, test',
 'task': 'readmission',
 'seed': 1,
 'ratio': '100',
 'n_epochs': 1000,
 'lr': 0.0001,
 'batch_size': 128,
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'rnn_layer': 1,
 'dropout': 0.3,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'load_pretrained_weights': 'None',
 'transfer': 'None',
 'model': 'None',
 'embed_model': 'None',
 'pred_model': 'None',
 'bert_model': 'bert_tiny',
 'init_bert_params': 'None',
 'init_bert_params_with_freeze': 'None'}

In [3]:
class Argument(object):
    def __init__(self, arg_dict:dict):
        super().__init__()
        for key, value in arg_dict.items():
            setattr(self, key, value)
            
    def __repr__(self):
        return str(self.__dict__)

In [4]:
args = Argument(arg_dict)

In [5]:
args

{'distributed_world_size': 1, 'input_path': 'None', 'model_path': 'None', 'save_dir': '../project_DLH/checkpoints', 'save_prefix': 'checkpoint', 'patience': 5, 'disable_validation': 'None', 'src_data': 'None', 'eval_data': 'None', 'value_mode': 'NV', 'fold': 'None', 'valid_subsets': 'valid, test', 'task': 'readmission', 'seed': 1, 'ratio': '100', 'n_epochs': 1000, 'lr': 0.0001, 'batch_size': 128, 'enc_embed_dim': 128, 'enc_hidden_dim': 256, 'rnn_layer': 1, 'dropout': 0.3, 'pred_embed_dim': 128, 'pred_hidden_dim': 256, 'max_event_len': 150, 'mlm_prob': 0.3, 'load_pretrained_weights': 'None', 'transfer': 'None', 'model': 'None', 'embed_model': 'None', 'pred_model': 'None', 'bert_model': 'bert_tiny', 'init_bert_params': 'None', 'init_bert_params_with_freeze': 'None'}

In [6]:
from DescEmb.trainers import Word2VecTrainer

ModuleNotFoundError: No module named 'utils'